In [1]:
import glob
import os
import numpy as np
import pickle
import math as mt
import shutil

In [9]:
def create_windows_antennas(csi_list, labels_list, sample_length, stride_length, remove_mean=False):
    csi_matrix_stride = []
    labels_stride = []
    for i in range(len(labels_list)):
        csi_i = csi_list[i]
        label_i = labels_list[i]
        len_csi = csi_i.shape[2]
        for ii in range(0, len_csi - sample_length, stride_length):
            csi_wind = csi_i[:, :, ii:ii + sample_length, ...]
            if remove_mean:
                csi_mean = np.mean(csi_wind, axis=2, keepdims=True)
                csi_wind = csi_wind - csi_mean
            csi_matrix_stride.append(csi_wind)
            labels_stride.append(label_i)
    return csi_matrix_stride, labels_stride

In [10]:
def convert_to_number(lab, csi_label_dict):
    lab_num = np.argwhere(np.asarray(csi_label_dict) == lab)[0][0]
    return lab_num

In [11]:
dir_init = './doppler_traces/' #Directory of data
subdir_init = 'S1a,S1b,S1c' #Sub-directories
sample_lengths_init = 31 #Number of packets in a sample
sliding_init = 1 #Number of packet for sliding operations
windows_length_init = 340 #Number of samples per window
stride_lengths_init = 30 #Number of samples to stride
labels_activities_init = 'E,L,W,R,J' #Labels of the activities to be considered
n_tot_init = 4 #Number of streams * number of antennas

In [12]:
labels_activities = labels_activities_init
csi_label_dict = []
for lab_act in labels_activities.split(','):
    csi_label_dict.append(lab_act)

activities = np.asarray(labels_activities)

In [13]:
n_tot = n_tot_init
num_packets = sample_lengths_init  # 51
middle = int(np.floor(num_packets / 2))
list_subdir = subdir_init  # string

In [14]:
for subdir in list_subdir.split(','):
    exp_dir = dir_init + subdir + '/'

    path_train = exp_dir + 'train_antennas_' + str(activities)
    path_val = exp_dir + 'val_antennas_' + str(activities)
    path_test = exp_dir + 'test_antennas_' + str(activities)
    paths = [path_train, path_val, path_test]
    for pat in paths:
        if os.path.exists(pat):
            remove_files = glob.glob(pat + '/*')
            for f in remove_files:
                os.remove(f)
        else:
            os.mkdir(pat)

    path_complete = exp_dir + 'complete_antennas_' + str(activities)
    if os.path.exists(path_complete):
        shutil.rmtree(path_complete)

    names = []
    all_files = os.listdir(exp_dir)
    for i in range(len(all_files)):
        if all_files[i].startswith('S'):
            names.append(all_files[i][:-4])
    names.sort()

    csi_matrices = []
    labels = []
    lengths = []
    label = 'null'
    prev_label = label
    csi_matrix = []
    processed = False
    for i_name, name in enumerate(names):
        if i_name % n_tot == 0 and i_name != 0 and processed:
            ll = csi_matrix[0].shape[1]

            for i_ant in range(1, n_tot):
                if ll != csi_matrix[i_ant].shape[1]:
                    break
            lengths.append(ll)
            csi_matrices.append(np.asarray(csi_matrix))
            labels.append(label)
            csi_matrix = []

        label = name[4]

        if label not in csi_label_dict:
            processed = False
            continue
        processed = True

        print(name)

        label = convert_to_number(label, csi_label_dict)
        if i_name % n_tot == 0:
            prev_label = label
        elif label != prev_label:
            print('error in ' + str(name))
            break

        name_file = exp_dir + name + '.txt'
        with open(name_file, "rb") as fp:  # Unpickling
            stft_sum_1 = pickle.load(fp)

        stft_sum_1_mean = stft_sum_1 - np.mean(stft_sum_1, axis=0, keepdims=True)

        csi_matrix.append(stft_sum_1_mean.T)

    error = False
    if processed:
        # for the last block
        if len(csi_matrix) < n_tot:
            print('error in ' + str(name))
        ll = csi_matrix[0].shape[1]

        for i_ant in range(1, n_tot):
            if ll != csi_matrix[i_ant].shape[1]:
                print('error in ' + str(name))
                error = True
        if not error:
            lengths.append(ll)
            csi_matrices.append(np.asarray(csi_matrix))
            labels.append(label)

    if not error:
        lengths = np.asarray(lengths)
        length_min = np.min(lengths)

        csi_train = []
        csi_val = []
        csi_test = []
        length_train = []
        length_val = []
        length_test = []
        for i in range(len(labels)):
            ll = lengths[i]
            train_len = int(np.floor(ll * 0.6))
            length_train.append(train_len)
            csi_train.append(csi_matrices[i][:, :, :train_len])

            start_val = train_len + mt.ceil(num_packets/sliding_init)
            val_len = int(np.floor(ll * 0.2))
            length_val.append(val_len)
            csi_val.append(csi_matrices[i][:, :, start_val:start_val + val_len])

            start_test = start_val + val_len + mt.ceil(num_packets/sliding_init)
            length_test.append(ll - val_len - train_len - 2*mt.ceil(num_packets/sliding_init))
            csi_test.append(csi_matrices[i][:, :, start_test:])

        window_length = windows_length_init  # number of windows considered
        stride_length = stride_lengths_init

        list_sets_name = ['train', 'val', 'test']
        list_sets = [csi_train, csi_val, csi_test]
        list_sets_lengths = [length_train, length_val, length_test]

        for set_idx in range(3):
            csi_matrices_set, labels_set = create_windows_antennas(list_sets[set_idx], labels, window_length,
                                                                    stride_length, remove_mean=False)

            num_windows = np.floor((np.asarray(list_sets_lengths[set_idx]) - window_length) / stride_length + 1)
            if not len(csi_matrices_set) == np.sum(num_windows):
                print('ERROR - shapes mismatch')
                print(len(csi_matrices_set), ' and ', np.sum(num_windows), '\n')

            names_set = []
            suffix = '.txt'
            for ii in range(len(csi_matrices_set)):
                name_file = exp_dir + list_sets_name[set_idx] + '_antennas_' + str(activities) + '/' + \
                            str(ii) + suffix
                names_set.append(name_file)
                with open(name_file, "wb") as fp:  # Pickling
                    pickle.dump(csi_matrices_set[ii], fp)
            name_labels = exp_dir + '/labels_' + list_sets_name[set_idx] + '_antennas_' + str(activities) + suffix
            with open(name_labels, "wb") as fp:  # Pickling
                pickle.dump(labels_set, fp)
            name_f = exp_dir + '/files_' + list_sets_name[set_idx] + '_antennas_' + str(activities) + suffix
            with open(name_f, "wb") as fp:  # Pickling
                pickle.dump(names_set, fp)
            name_f = exp_dir + '/num_windows_' + list_sets_name[set_idx] + '_antennas_' + str(activities) + suffix
            with open(name_f, "wb") as fp:  # Pickling
                pickle.dump(num_windows, fp)


S1a_E_stream_0
S1a_E_stream_1
S1a_E_stream_2
S1a_E_stream_3
S1a_J1_stream_0
S1a_J1_stream_1
S1a_J1_stream_2
S1a_J1_stream_3
S1a_J2_stream_0
S1a_J2_stream_1
S1a_J2_stream_2
S1a_J2_stream_3
S1a_L_stream_0
S1a_L_stream_1
S1a_L_stream_2
S1a_L_stream_3
S1a_R_stream_0
S1a_R_stream_1
S1a_R_stream_2
S1a_R_stream_3
S1a_W_stream_0
S1a_W_stream_1
S1a_W_stream_2
S1a_W_stream_3
S1b_E_stream_0
S1b_E_stream_1
S1b_E_stream_2
S1b_E_stream_3
S1b_J1_stream_0
S1b_J1_stream_1
S1b_J1_stream_2
S1b_J1_stream_3
S1b_J2_stream_0
S1b_J2_stream_1
S1b_J2_stream_2
S1b_J2_stream_3
S1b_L_stream_0
S1b_L_stream_1
S1b_L_stream_2
S1b_L_stream_3
S1b_R_stream_0
S1b_R_stream_1
S1b_R_stream_2
S1b_R_stream_3
S1b_W_stream_0
S1b_W_stream_1
S1b_W_stream_2
S1b_W_stream_3
ERROR - shapes mismatch
1799  and  1800.0 

ERROR - shapes mismatch
556  and  557.0 

ERROR - shapes mismatch
543  and  545.0 

S1c_E_stream_0
S1c_E_stream_1
S1c_E_stream_2
S1c_E_stream_3
S1c_J1_stream_0
S1c_J1_stream_1
S1c_J1_stream_2
S1c_J1_stream_3
S1c_J2_strea